# Test 1

### Import

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from backtesting import Backtest
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
from datetime import datetime
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

### Device

In [11]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using device: {device}")

Using device: cpu


### Hyperparameter

In [12]:
# Model parameter
input_size = 25
output_size = 1
hidden_size = 1000
num_layers = 6
dropout = 0.2

# Training parameter
batch_size = 16
num_epochs = 0
learning_rate = 0.001
seq_size = 30

### LSTM Model

In [13]:
class Net(nn.Module):

    def __init__(self, input_size, output_size, hidden_size, num_layers):
        super(Net, self).__init__()
        self.layer_1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.layer_2 = nn.LSTM(hidden_size, hidden_size, batch_first=True, dropout=dropout, num_layers=num_layers)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.layer_1(x)  
        x = torch.relu(x)

        x, _ = self.layer_2(x)
        x = torch.relu(x)

        x = self.output_layer(x[:, -1, :])  
        
        return x

### Dataloader

In [14]:
class FinanceDataset(Dataset):
    def __init__(self, data_path, seq_size):
        self.data = pd.read_pickle(data_path)
        print(self.data)
        self.seq_size = seq_size
        
        self.data = self.data[['^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume', 'GC=F_month', 'GC=F_weekday',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume', 'BZ=F_month', 'BZ=F_weekday',
                                '^GDAXI_trend', 'Invest']]

        # Entfernen der 'Date'-Spalte für Inputs
        self.inputs = self.data.drop(columns=['Invest']).values
        self.labels = self.data['Invest'].values  # Die Label-Spalte ist hier 'Invest'

        # Skalieren der Eingabedaten
        scaler = MinMaxScaler()
        self.inputs = scaler.fit_transform(self.inputs)

    def __len__(self):
        return len(self.inputs) - self.seq_size

    def __getitem__(self, idx):
        # Slice für Sequenzeingaben und das entsprechende Label
        x = self.inputs[idx:idx + self.seq_size]
        y = self.labels[idx + self.seq_size]

        # In Tensoren umwandeln
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        return x, y

### Init

In [15]:
# Initialize model, loss function, optimizer
net = Net(input_size, output_size, hidden_size, num_layers)
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)

In [16]:
# Initialize dataset and dataloader
dataset = FinanceDataset('../Data/train_dax_data.pkl', seq_size=seq_size)
train_loader = DataLoader(dataset, batch_size=batch_size)

           Date   ^GDAXI_Open   ^GDAXI_High    ^GDAXI_Low  ^GDAXI_Close  \
0    2015-01-02   9869.129883   9879.530273   9687.259766   9764.730469   
1    2015-01-05   9735.650391   9790.269531   9468.580078   9473.160156   
2    2015-01-06   9484.250000   9624.650391   9382.820312   9469.660156   
3    2015-01-07   9510.339844   9592.370117   9459.179688   9518.179688   
4    2015-01-08   9643.769531   9855.429688   9607.900391   9837.610352   
...         ...           ...           ...           ...           ...   
1970 2022-12-23  13945.589844  14000.679688  13874.500000  13940.929688   
1971 2022-12-27  14047.419922  14063.139648  13966.349609  13995.099609   
1972 2022-12-28  14013.719727  14018.469727  13914.620117  13925.599609   
1973 2022-12-29  13890.809570  14071.719727  13871.320312  14071.719727   
1974 2022-12-30  14005.839844  14008.969727  13922.549805  13923.589844   

      ^GDAXI_Adj Close  ^GDAXI_Volume  ^GDAXI_month  ^GDAXI_weekday Symbol_x  \
0          9764.730

### Training

In [17]:
os.makedirs("../Models", exist_ok=True)
losses = []
# Training loop
for epoch in range(num_epochs):
    running_loss = 0
    for inputs, labels in train_loader:
        #print(input)
        optimizer.zero_grad()
        outputs = net(inputs.float())
        loss = criterion(torch.squeeze(outputs), labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Save model after each epoch
    model_path = f'../Models/model-{epoch + 1}.pt'
    torch.save(net.state_dict(), model_path)

    
    if epoch % 10 == 0:
        learning_rate *= 0.8
        optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, {running_loss}, {train_loader}')
    losses.append(running_loss / len(train_loader))

### Backtesting

In [ ]:
os.makedirs("logs", exist_ok=True)
os.makedirs("results", exist_ok=True)

scaler = MinMaxScaler()
df = pd.read_pickle('../Data/train_dax_data.pkl').reset_index()
df = df[['^GDAXI_Open', '^GDAXI_High', '^GDAXI_Low', '^GDAXI_Close',
                                '^GDAXI_Adj Close', '^GDAXI_Volume', '^GDAXI_month', '^GDAXI_weekday',
                                'GC=F_Open', 'GC=F_High', 'GC=F_Low', 'GC=F_Close',
                                'GC=F_Adj Close', 'GC=F_Volume', 'GC=F_month', 'GC=F_weekday',
                                'BZ=F_Open', 'BZ=F_High', 'BZ=F_Low', 'BZ=F_Close',
                                'BZ=F_Adj Close', 'BZ=F_Volume', 'BZ=F_month', 'BZ=F_weekday',
                                '^GDAXI_trend']]
scaler.fit(df.values)

models = [f for f in os.listdir("../Models")]
alredy_done = [f.removeprefix("backtest-").removesuffix(".csv.gz") for f in os.listdir("results/")]
print(len(models))
print(len(alredy_done))

test_data = pd.read_pickle("../Data/test_dax_data.pkl")

for model_name in models:
    # Skip Backtesting for Models which already did the Backtesting
    if model_name in alredy_done:
        print(f"Skip model: {model_name}")
        continue

    model_path = f"../Models/{model_name}"
    model = Net(input_size, output_size, hidden_size, num_layers)

    # Load state_dict only
    model.load_state_dict(torch.load(model_path, weights_only=False))
    model.eval()

    ALPACA_CREDS = {
        "API_KEY": os.getenv("ALPACA_API_KEY"), 
        "API_SECRET": os.getenv("ALPACA_API_SECRET"), 
        "PAPER": True
    }
    # Strategy setup
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    broker = Alpaca(ALPACA_CREDS)

    # Instantiate and run the strategy
    strategy = Backtest(
        name=model_name,  
        broker=broker,
        parameters={
            "symbol": "^GDAXI",
            "cash_at_risk": 0.5,
            "model": model,
            "num_prior_days": 30,
            "dataset": test_data,
            "scaler": scaler,
        }
    )

    # Run the backtest
    backtest_results = strategy.backtest(
        YahooDataBacktesting,
        start_date,
        end_date,
        name=model_name,
        parameters={
            "symbol": "^GDAXI", 
            "cash_at_risk": 0.5, 
            "model": model,  
            "dataset": test_data,
            "num_prior_days": 30,
            "scaler": scaler,
        },
        benchmark_asset="^GDAXI",
        show_plot=True,
        show_tearsheet=True
    )

    # Convert to DataFrame only if results are non-empty
    backtest_results = pd.DataFrame(backtest_results)   
    backtest_results["model"] = model_name
    
    backtest_results.to_csv(f"results/backtest-{model_name}.csv.gz", index=False, compression='gzip')
    
print("Backtesting complete. Results saved to backtest_results.csv.")

87
8
Skip model: model-1.pt
Skip model: model-10.pt
Skip model: model-11.pt
Skip model: model-12.pt
Skip model: model-13.pt
Skip model: model-14.pt
Skip model: model-15.pt
Skip model: model-16.pt
Starting backtest for Backtest...
Progress ||   1.21%  [Elapsed: 0:00:01 ETA: 0:01:44] Portfolio Val: 100,737.00

c:\Users\wistudent\Documents\AKI-Trading-Model\Test4-Raw Material\Code\backtesting.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Progress || 100.00%  [Elapsed: 0:00:25 ETA: 0:00:00] Portfolio Val: 104,677.35
Creating trades plot...

Creating indicators plot...

Creating tearsheet...
Starting backtest for Backtest...
Progress ||   1.48%  [Elapsed: 0:00:01 ETA: 0:01:45] Portfolio Val: 100,762.49

c:\Users\wistudent\Documents\AKI-Trading-Model\Test4-Raw Material\Code\backtesting.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Progress || 100.00%  [Elapsed: 0:00:26 ETA: 0:00:00] Portfolio Val: 104,677.35
Creating trades plot...

Creating indicators plot...

Creating tearsheet...
Starting backtest for Backtest...
Progress ||   1.21%  [Elapsed: 0:00:01 ETA: 0:01:37] Portfolio Val: 100,737.00

...

Progress || 100.00%  [Elapsed: 0:00:30 ETA: 0:00:00] Portfolio Val: 104,677.35
Creating trades plot...

Creating indicators plot...

Creating tearsheet...
Starting backtest for Backtest...
Progress ||   1.21%  [Elapsed: 0:00:01 ETA: 0:01:41] Portfolio Val: 100,737.00

c:\Users\wistudent\Documents\AKI-Trading-Model\Test4-Raw Material\Code\backtesting.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Progress || 100.00%  [Elapsed: 0:00:30 ETA: 0:00:00] Portfolio Val: 104,677.35
Creating trades plot...

Creating indicators plot...

Creating tearsheet...
Starting backtest for Backtest...
Progress ||   1.21%  [Elapsed: 0:00:01 ETA: 0:01:53] Portfolio Val: 100,737.00

c:\Users\wistudent\Documents\AKI-Trading-Model\Test4-Raw Material\Code\backtesting.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Progress || 100.00%  [Elapsed: 0:00:31 ETA: 0:00:00] Portfolio Val: 104,677.35
Creating trades plot...

Creating indicators plot...

Creating tearsheet...


In [ ]:
df = pd.read_csv("results.csv")
display(df.sort_values(by=["model"]))

FileNotFoundError: [Errno 2] No such file or directory: 'results.csv'

In [ ]:

best_model = df[df["total_return"] == df["total_return"].max()]
best_model.reset_index(inplace=True)

model_name = best_model.at[0, "model"]
print(f"Best model: {model_name}")

In [ ]:
import pandas as pd
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
import numpy as np

model_path = f"../Models/{model_name}"
model = Net(input_size, output_size, hidden_size, num_layers)

# Load state_dict only
model.load_state_dict(torch.load(model_path))  # Do not use weights_only
model.eval()

# Load your DataFrame
data = pd.read_pickle('../Data/test_dax_data.pkl')

test_data = FinanceDataset('../Data/test_dax_data.pkl', seq_size=seq_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
# Select all rows and columns from index 1 to -1 (exclusive)

all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        out = model(inputs) 
        # Store predictions and labels
        all_predictions.append(out.numpy())  
        all_labels.append(labels.numpy())

# Concatenate results
all_predictions = np.concatenate(all_predictions)
all_labels = np.concatenate(all_labels)

# Print or analyze the predictions
print(f'Predicted values: {all_predictions.flatten()}')
print(f'Actual values: {all_labels.flatten()}')

output_df = pd.DataFrame({'Predicted': all_predictions.flatten(), 'Actual': all_labels.flatten()})


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

threshold = 0.5
predicted_classes = (all_predictions.flatten() > threshold).astype(int)

accuracy = accuracy_score(all_labels.flatten(), predicted_classes)
precision = precision_score(all_labels.flatten(), predicted_classes)
recall = recall_score(all_labels.flatten(), predicted_classes)
f1 = f1_score(all_labels.flatten(), predicted_classes)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(all_labels.flatten(), predicted_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
output_df = pd.DataFrame({
    'Predicted': all_predictions.flatten(),
    'Predicted_Class': predicted_classes,
    'Actual': all_labels.flatten()
})
correlation_matrix = output_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()
display(output_df)